# Validation - BAC response of models

This notebook evaluates qualitatively the capability of fitted models to reproduce BAC behavior.

In [ ]:
import json
import sys
import os
import time
import neuron
from pathlib import Path
import pickle
import pandas as pd

import matplotlib.pyplot as plt
import neuroplotlib as nplt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import LFPy
import neuron
import numpy as np

import shutil

import model
import evaluator
import plotting
import utils

# %matplotlib notebook
%matplotlib widget

In [ ]:
probe_type = "planar"
model_name = "hay_ais" #'hallermann' # "hay" hay_ais
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [ ]:
data_folder = Path('..') / "data" / f"{model_name}_ecode_probe_{probe_type}"
result_folder = Path("..") / "results" / 'models' / 'opt_results' 

In [ ]:
# compile model-specific mechanisms
if not (model_folder / "x86_64").is_dir():
    curr_dir = Path(".").absolute()
    os.chdir(model_folder)
    print(os.getcwd())
    os.system("nrnivmodl mechanisms")
    os.chdir(curr_dir)
else:
    print(f"Mechanisms already compiled for {model_name}")
    compiled_folder = Path("./x86_64")
    if compiled_folder.is_dir():
        shutil.rmtree(compiled_folder)
    shutil.copytree(model_folder / "x86_64", compiled_folder)

In [ ]:
data = pickle.load(open(result_folder / 'runs.pkl', 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == '{model_name}'")

In [ ]:
opt_soma = df_model.query("feature_set == 'soma'")
opt_extra = df_model.query("feature_set == 'extra'")
print(f"Somatic optimizations: {len(opt_soma)}")
print(f"Extra optimizations: {len(opt_extra)}")

In [ ]:
protocols_file = data_folder / "efeatures/protocols_BPO.json"
features_file = data_folder /  "efeatures/features_BPO.json"

In [ ]:
ais_recordings = utils.get_ais_extra_recordings()

In [ ]:
protocols_file

In [ ]:
cell = model.create(model_name=model_name, release=False)
cell_release = model.create(model_name=model_name, release=True)

probe = model.define_electrode(probe_type=probe_type)
# probe=None

param_names = [param.name for param in cell.params.values() if not param.frozen]
sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe)

fitness_protocols = evaluator.define_protocols(
    model_name,
    feature_set="soma",
    feature_file=features_file,#f"{model_name}_model/features_list.json",
    protocols_file=protocols_file,
    electrode=probe,
    protocols_with_lfp=["firepattern_200"],
    extra_recordings=dict(firepattern_200=ais_recordings)  #["Step1", "bAP", "BAC", "EPSP", "CaBurst" ]
)

params_release = {}
for param in cell_release.params_by_names(param_names):
    params_release[param.name] = param.value

In [ ]:
params_release

# Test AIS dynamics

In [ ]:
fitness_protocols

In [ ]:
def simulate_firepattern_responses(cell, params, sim, protocols, protocol_name="firepattern_200"):
    response = protocols[protocol_name].run(cell, params, sim)
    eap = utils.calculate_eap(responses=responses, protocols=eva_extra.fitness_protocols, 
                              protocol_name="firepattern_200", **extra_kwargs)
    return response, eap

In [ ]:
response_release = simulate_firepattern_responses(cell, params_release, sim, fitness_protocols)

print("Computing SOMA")
responses_soma = []
eaps_soma = []
for i, (index, opt) in enumerate(opt_soma.iterrows()):
    print(f"Computing {i + 1} / {len(opt_soma)}")
    params_dict = {k: v for k, v in zip(param_names, opt.best_params)}
    response, eap = simulate_firepattern_responses(cell, params_dict, sim, fitness_protocols)
    responses_soma.append(response)
    eaps_soma.append(eap)

print("Computing EXTRA")
responses_extra = []
eaps_extra = []
for i, (index, opt) in enumerate(opt_extra.iterrows()):
    print(f"Computing {i + 1} / {len(opt_soma)}")
    params_dict = {k: v for k, v in zip(param_names, opt.best_params)}
    response, eap = simulate_firepattern_responses(cell, params_dict, sim, fitness_protocols)
    responses_extra.append(response)
    eaps_extra.append(eap)

In [ ]:
ax_dict = {}

loc_names = resp_release.keys()

for loc in loc_names:
    fig, ax = plt.subplots()
    ax_dict[loc] = ax

    for i, resp in enumerate(responses_soma):
        if i == 0:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C0", label="soma")
        else:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C0")

    for i, resp in enumerate(responses_extra):
        if i == 0:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C1", label="extra")
        else:
            ax.plot(resp[loc]["time"], resp[loc]["voltage"], alpha=0.4, color="C1")

    ax.plot(resp_release[loc]["time"], resp_release[loc]["voltage"], alpha=0.8, color="k", lw=2, 
            label="GT")
    ax.set_title(f"Recording from {loc}")
    ax.legend()